# IR on Time documents collection

IMPORTING PACKAGES

In [1]:
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
from nltk import FreqDist
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
import operator

READING IN THE DOCUMENTS

In [2]:
query=open("TIME.QUE","r",encoding='utf-8-sig').read().split('*')
text=open("TIME.ALL","r",encoding='utf-8-sig').read().split('*')
rel=pd.read_csv('REL.csv',header=None)  
stop=open("TIME.STP","r",encoding='utf-8-sig').read().split('\n\n')
numbers=list(range(0,450))
new_stop=[]
for i in numbers:
    k=str(i)
    new_stop.append(k)
for i in stop:
    i=i.lower()
    new_stop.append(i)

INPUT AND OUTPUT

In [10]:
####READING IN REAL TIME ####
def show():
    r=int(input('query index'))
    term_doc=term_weights(text)
    q=query_process(query[r])
    docs=retrieve(q)
    keys=list(docs.keys())
    top10=keys[:11]
    for i in top10:
        print (text[i])

In [11]:
show()

query index1
TEXT 071 02/08/63 PAGE 022

EUROPE A NEW & OBSCURE DESTINATION

IN AN ALLIANCE IN WHICH PARTNERS HAD BECOME INCREASINGLY

MINDFUL OF ONE ANOTHER'S SENSITIVITIES, IN WHICH VICTORIES WERE

TACTFULLY NOT CROWED OVER, AND TOGETHERNESS IN ITSELF WAS REGARDED AS A

GOOD THING, CHARLES DE GAULLE LAST WEEK REMINDED THE WORLD OF WHAT ONE

MAN WITH A WILL CAN DO . OVER THE PAST 20 YEARS, ON HIS OWN PEOPLE AND

ON THEIR NEIGHBORS, CHARLES DE GAULLE HAS PERFECTED HIS NATIVE TALENT

FOR HANDING OUT RUDE SURPRISES . EMPLOYING THE LOFTY DISDAIN THAT USED

TO INFURIATE ROOSEVELT, CHURCHILL AND STALIN, DE GAULLE SUCCEEDED IN

INFURIATING KENNEDY, MACMILLAN AND A HOST OF OTHERS BY VETOING

BRITAIN'S ADMISSION TO THE COMMON MARKET . HE MIGHT HAVE BEEN GENTLE

ABOUT IT, ALLOWING NEGOTIATIONS TO SPUTTER OUT IN A CONFUSION OF

DETAILS . HE CHOSE TO BE BLUNT AND HIS PURPOSE WAS NOT JUST TO SHUT

OUT BRITAIN OR MERELY TO HUMILIATE HER . HE ALSO SHOOK THE ATLANTIC

ALLIANCE TO ITS ROOTS, SET OUT T

In [12]:
all_res=pd.read_csv('final.csv',header=None)  

In [13]:
#### RETRIEVING FROM OFFLINE RESULTS ####
def show2():
    r=int(input('query index'))
    docs=all_res.iloc[r]
    keys=docs.tolist()
    for i in keys:
        print (text[i])

In [14]:
show2()

query index0
TEXT 071 02/08/63 PAGE 022

EUROPE A NEW & OBSCURE DESTINATION

IN AN ALLIANCE IN WHICH PARTNERS HAD BECOME INCREASINGLY

MINDFUL OF ONE ANOTHER'S SENSITIVITIES, IN WHICH VICTORIES WERE

TACTFULLY NOT CROWED OVER, AND TOGETHERNESS IN ITSELF WAS REGARDED AS A

GOOD THING, CHARLES DE GAULLE LAST WEEK REMINDED THE WORLD OF WHAT ONE

MAN WITH A WILL CAN DO . OVER THE PAST 20 YEARS, ON HIS OWN PEOPLE AND

ON THEIR NEIGHBORS, CHARLES DE GAULLE HAS PERFECTED HIS NATIVE TALENT

FOR HANDING OUT RUDE SURPRISES . EMPLOYING THE LOFTY DISDAIN THAT USED

TO INFURIATE ROOSEVELT, CHURCHILL AND STALIN, DE GAULLE SUCCEEDED IN

INFURIATING KENNEDY, MACMILLAN AND A HOST OF OTHERS BY VETOING

BRITAIN'S ADMISSION TO THE COMMON MARKET . HE MIGHT HAVE BEEN GENTLE

ABOUT IT, ALLOWING NEGOTIATIONS TO SPUTTER OUT IN A CONFUSION OF

DETAILS . HE CHOSE TO BE BLUNT AND HIS PURPOSE WAS NOT JUST TO SHUT

OUT BRITAIN OR MERELY TO HUMILIATE HER . HE ALSO SHOOK THE ATLANTIC

ALLIANCE TO ITS ROOTS, SET OUT T

PREPROCESSING FUNCTIONS

In [3]:
#### STEP 1---TOKENIZATION
def tokenization(f):
    f = f.lower()
    f=f.replace('\n',' ')
    tokens=word_tokenize(f)
    return tokens

def preprocessing(f):    
#### STEP 2----STOP WORDS AND PUNCTUATIONS REMOVAL ####
    stop_words = set(stopwords.words("english"))
    p={',','.','(',')','"','``',"'s","''",'%',"'",'\n',"'ll",'“','”','[',']',':','-','—',\
       '●','’',';','--','?','&','#','!','*','^','$','/'}
    removal=stop_words.union(p)
    filtered=[]
    for w in f:
        if w not in removal:
            filtered.append(w)   
#### STEP 3----LEMMATIZATION AND STEMMING ####
    lemmtizer = WordNetLemmatizer()
    lemmatized=[]
    for i in range(len(filtered)):
        words = word_tokenize(filtered[i])
        words = [lemmtizer.lemmatize(word) for word in words]
        lemmatized.append(' '.join(words))
    stemmer = PorterStemmer()
    new_sentence = []
    for i in range(len(lemmatized)):
        words = word_tokenize(lemmatized[i])
        words = [stemmer.stem(word) for word in words]
        new_sentence.append(' '.join(words))    
#### STEP 4----NORMALIZED TERM FREQUENCY ####
    norm_freq={}
    freq = FreqDist(new_sentence)
    for i in freq:
        t=round(freq[i]/len(freq),3)
        norm_freq[i]=t       
    return norm_freq

TFIDF FUNCTIONS

In [4]:
#### STEP 1----DOCUMENT FREQUENCY FOR EACH TERM ####
def df(y):
    'y is the list of terms from each document'
    token_count= dict()
    for token in y:
        if token in token_count:
            token_count[token] += 1
        else:
            token_count[token] = 1
    return token_count

In [5]:
#### STEP 2----TFIDF WEIGHTS FOR EACH TERM ####
def TFIDF(y,doc,N):
    'y is a DF,doc is the document, \
    N is the number of docs in the collection'
    'calculate IDF for all unique terms'
    idf=dict()
    for i in y and doc:
        w=round(np.log2(N/y[i]),3)
        idf[i]=w
    tfidf=dict()
    for k in idf.keys():
        if k in doc.keys():
            tid=round(idf[k]*doc[k],3)
            tfidf[k]=tid
    return tfidf

In [6]:
#### STEP 3---- CALL ABOVE FUNCTIONS ON DOCUMENTS ####
def term_weights(f):
    'out the TFIDF weights for each term in the doc\
    doc is a dictionary of (term:tfidf value), and append\
    all doc dictionaries to a list of entire collection'
    articles=[]
    for i in f:
        article=[]
        token=tokenization(i)
        for w in token:
            if w not in new_stop:
                article.append(w)
        articles.append(article)
    
    clean_articles=[]
    for i in articles:
        clean_articles.append(preprocessing(i))
    
    weights=[]  
    all_terms=[]
    for w in clean_articles:
        terms=list(w.keys())
        all_terms.append(terms)
    flatList = [ item for elem in all_terms for item in elem]
    doc_freq=df(flatList)
    
    for i in clean_articles:
        weight=TFIDF(doc_freq,i,423)
        weights.append(weight) #TFIDF weights of terms in docs
    return weights

QUERY PROCESSING

In [7]:
def query_process(y):
    'y is a query, ouput tokenized query term and its raw frequency'
    tokens=[]
    clean_token=[]
    query_token=y.split()
    for i in query_token:
        i=i.lower()
        i=i.replace('\n',' ')
        tokens.append(i)
    for w in tokens:
            if w not in new_stop:
                clean_token.append(w)  
    clean_query=preprocessing(clean_token)
    return clean_query

VECTOR SPACE RETRIEVAL MODEL

In [8]:
def retrieve(q):
    'function takes in a query vector, retrived documents,\
    ranks doc relevance based on cosine similarity and \
    output retrieved docs and its relevance score'
    retrieved=[]
    related=[]
    res=dict()
    scores=dict()
    for i in q.keys(): #retrieved doc that has at least one query term
        for x in range(len(term_doc)):
            each=term_doc[x]
            if i in each.keys():
                retrieved.append((x,i,each[i]))
    for x in range(len(term_doc)): #extract tfidf of the term
        each=term_doc[x]
        for i in q.keys():
            if i in each.keys():
                related.append([x,i,each[i]])
    for i in related:
        currentkey=i[0]
        if currentkey in res:
            res[currentkey].append(i[2])
        else:
            res[currentkey]=[i[2]]
    fill=len(q) #fill nulls with zeros
    for i in res:
        lnow=len(res[i])
        if lnow<fill:
            res[i].extend([0]*(fill-lnow))
    c=10**18 #reduce decimals
    for i in res:
        score=round(c*cosine(list(q.values()),i),3)
        scores[i]=score    
    ranked = dict(sorted(scores.items(), key=operator.itemgetter(1),reverse=True))
    return ranked

In [9]:
term_doc=term_weights(text)
doc_ind=[]
for i in range(len(query)):
    q=query_process(query[i])
    trial=retrieve(q)
    keys=list(trial.keys())
    l1=keys[:11]
    doc_ind.append(l1)

In [ ]:
doc_ind

In [ ]:
with open('list.txt', 'w') as filehandle:
    for listitem in doc_ind:
        filehandle.write('%s\n' % listitem)